## Laboratory 01 - Topological Data Analysis  
Package: giotto-tda

In [1]:
# install package giotto-tda
import sys
!{sys.executable} -m pip install -U giotto-tda

In [1]:
# import packages
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_circles
from pathlib import Path
import plotly.graph_objects as go

# import packages for topology tasks
from gtda.homology import VietorisRipsPersistence
from gtda.time_series import SingleTakensEmbedding
from gtda.diagrams import BettiCurve, PersistenceLandscape, PairwiseDistance, PersistenceEntropy
from gtda.plotting import plot_point_cloud, plot_diagram
from gtda.pipeline import Pipeline

1) Cloud of points

In [2]:
# generate cloud of points
np.random.seed(seed = 42)

x = np.asarray([
    make_circles(100, factor = np.random.random())[0]
    for i in range(10)
])

In [5]:
print(x.shape)
print(x[4].shape)

(10, 100, 2)
(100, 2)


In [6]:
i = 4
plot_point_cloud(x[i])

2) Vector Rips persistence

In [7]:
vr = VietorisRipsPersistence()
xt = vr.fit_transform(x)
vr.plot(xt, sample = i)

Betti curve and Betti value

In [8]:
bc = BettiCurve()
x_betti_curve = bc.fit_transform_plot(xt)    

Landscape persistence

In [9]:
pl = PersistenceLandscape()
x_persistence_landscape = pl.fit_transform_plot(xt)

Time Series

In [10]:
def basic_time_series(n, b, y):
    r = np.random.rand(n)
    j = 2 * np.pi + np.arcsin(r)
    t = np.arange(n)
    x = b * np.sin(y * t + j)
    return x

In [11]:
def noisy_time_series(n, b, y):
    r = np.random.rand(n)
    j = 2 * np.pi * r
    t = np.arange(n)
    x = b * np.sin(y * t + j)
    return x

In [12]:
n = 500
b = 1.5
y = 1

basic_ts = basic_time_series(n, b, y)
noisy_ts = noisy_time_series(n, b, y)

In [13]:
fig = go.Figure(data = go.Scatter(y = basic_ts))
fig.update_layout(xaxis_title = "Time-stamp", yaxis_title = "Amplitude")
fig.show()

In [14]:
fig = go.Figure(data = go.Scatter(y = noisy_ts))
fig.update_layout(xaxis_title = "Time-stamp", yaxis_title = "Amplitude")
fig.show()

In [15]:
embedding_dimension_periodic = 3
embedding_time_delay_periodic = 8

embedder = SingleTakensEmbedding(
    parameters_type = "fixed",
    time_delay = embedding_time_delay_periodic,
    dimension = embedding_dimension_periodic
)

basic_point_cloud = embedder.fit_transform(basic_ts)
noisy_point_cloud = embedder.fit_transform(noisy_ts)

In [16]:
plot_point_cloud(basic_point_cloud).show()

In [17]:
plot_point_cloud(noisy_point_cloud)

In [18]:
# 0 - connected components, 1 - loops, 2 - voids
homology_dimensions = [0, 1, 2]

persistence = VietorisRipsPersistence(
    homology_dimensions = homology_dimensions
)

In [19]:
diagrams = persistence.fit_transform([basic_point_cloud, noisy_point_cloud])

In [20]:
vr.plot(diagrams, sample = 0)

In [21]:
vr.plot(diagrams, sample = 1)

In [25]:
# calculate metric of wasserstein
pw = 2 
pd = PairwiseDistance(metric = "wasserstein", 
                      metric_params = {"p": pw, "delta": 0.1},
                      order = None)
ws_dis = pd.fit_transform(diagrams)
print(ws_dis)

[[[0.         0.         0.        ]
  [1.29016735 0.92262653 0.23460986]]

 [[1.2896347  0.92080579 0.23460986]
  [0.         0.         0.        ]]]


In [27]:
# dimensionality of wasserstein metric
print("dim wassertein-dis = ", ws_dis.shape)

dim wassertein-dis =  (2, 2, 3)


In [29]:
# calculate the metric of bottleneck
pd2 = PairwiseDistance(metric = "bottleneck",
                       metric_params = {"delta": 0.1},
                       order = None)
bottleneck_dis = pd2.fit_transform(diagrams)

In [30]:
# dimensionality of bottleneck metric
print("dim bottleneck-dis = ", bottleneck_dis.shape)

dim bottleneck-dis =  (2, 2, 3)


In [31]:
# calculate the betti curve
bc = BettiCurve()
betti_curve = bc.fit_transform(diagrams)

In [32]:
bc.plot(betti_curve, sample = 0)

In [35]:
# calculate the betti distance
pd3 = PairwiseDistance(metric = "betti",
                       metric_params = {"p": 1},
                       order = None)
betti_dis = pd3.fit_transform(diagrams)
print(betti_dis)

[[[ 0.          0.          0.        ]
  [27.95031904 11.09281855  1.68033309]]

 [[27.95031904 11.09281855  1.68033309]
  [ 0.          0.          0.        ]]]


In [36]:
# dimensionality of betti distance
print("dim betti-dis = ", betti_dis.shape)

dim betti-dis =  (2, 2, 3)


In [37]:
# build the persitence landscape
pl = PersistenceLandscape()
landscapes = pl.fit_transform(diagrams)

In [38]:
pl.plot(landscapes, sample = 0)

In [39]:
pl.plot(landscapes, sample = 1)

In [40]:
# calculate the persistence landscape distance
pd4 = PairwiseDistance(metric = "landscape", 
                       metric_params = {"p": 1},
                       order = None)
landscape_dis = pd.fit_transform(diagrams)
print(landscape_dis)

[[[0.         0.         0.        ]
  [1.29016735 0.92262653 0.23460986]]

 [[1.2896347  0.92080579 0.23460986]
  [0.         0.         0.        ]]]


In [41]:
# check the dimensionality of landscape distance
print("dim landscape-dis = ", landscape_dis.shape)

dim landscape-dis =  (2, 2, 3)


Machine Learning task

In [42]:
def make_point_clouds(n_samples_per_shape: int, n_points: int, noise: float):
    circle_point_clouds = [
        np.asarray(
            [
                [np.sin(t) + noise * (np.random.rand(1)[0] - 0.5),
                 np.cos(t) + noise * (np.random.rand(1)[0] - 0.5), 0]
                 for t in range((n_points ** 2))
            ]
        )
        for kk in range(n_samples_per_shape)
    ]

    # label cicles with 0
    circle_labels = np.zeros(n_samples_per_shape)

    sphere_point_clouds = [
        np.asarray(
            [
                [np.cos(s) * np.cos(t) + noise * (np.random.rand(1)[0] - 0.5),
                 np.cos(s) * np.sin(t) + noise * (np.random.rand(1)[0] - 0.5),
                 np.sin(s) + noise * (np.random.rand(1)[0] - 0.5)]
                 for t in range(n_points)
                 for s in range(n_points)
            ]
        )
        for kk in range(n_samples_per_shape)
    ]

    # label spheres with 1
    sphere_labels = np.ones(n_samples_per_shape)

    torus_point_clouds = [
        np.asarray(
            [
                [(2 + np.cos(s)) * np.cos(t) + noise * (np.random.rand(1)[0] - 0.5),
                 (2 + np.cos(s)) * np.sin(t) + noise * (np.random.rand(1)[0] - 0.5),
                 np.sin(s) + noise * (np.random.rand(1)[0] - 0.5)]
                 for t in range(n_points)
                 for s in range(n_points)
            ]
        )
        for kk in range(n_samples_per_shape)
    ]

    # label tori with 2
    torus_labels = 2 * np.ones(n_samples_per_shape)

    point_clouds = np.concatenate((circle_point_clouds, sphere_point_clouds, torus_point_clouds))
    labels = np.concatenate((circle_labels, sphere_labels, torus_labels))

    return point_clouds, labels

In [44]:
# generate the dataset
point_clouds_basic, labels_basic = make_point_clouds(
                                        n_samples_per_shape = 10, 
                                        n_points = 20, 
                                        noise = 0.5)
print("dim point-cloud = ", point_clouds_basic.shape)
print("dim labels = ", labels_basic.shape)

dim point-cloud =  (30, 400, 3)
dim labels =  (30,)


In [45]:
plot_point_cloud(point_clouds_basic[0])

In [46]:
plot_point_cloud(point_clouds_basic[10])

In [47]:
plot_point_cloud(point_clouds_basic[-1])

In [48]:
# calculate the persistence homology
homology_dimensions = [0, 1, 2]

persistence_homology = VietorisRipsPersistence(
    metric = "euclidean",
    homology_dimensions = homology_dimensions, 
    collapse_edges = True
)

In [49]:
# generate the persistence diagram
diagrams_basic = persistence_homology.fit_transform(point_clouds_basic)

In [51]:
# plot the persistence diagram - circle
plot_diagram(diagrams_basic[0])

In [55]:
# plot the persistence diagram - spheres
plot_diagram(diagrams_basic[1])

In [56]:
# plot the persistence diagrams - tori
plot_diagram(diagrams_basic[2])

In [63]:
# check dimensionality of the persistence diagram
print("dim persistence-diagrams = ", diagrams_basic.shape)

dim persistence-diagrams =  (30, 599, 3)


In [57]:
# calculate the persistence entropy
persistence_entropy = PersistenceEntropy()

x_basic = persistence_entropy.fit_transform(diagrams_basic)

In [58]:
# plot the results of the persistence entropy
plot_point_cloud(x_basic)

In [61]:
# take a look inside of x_basic
print(x_basic)
print("dim x-basic = ", x_basic.shape)
print("labels-basic = ", labels_basic.shape)

[[8.48814251 4.61371778 0.47624779]
 [8.46209274 4.67434089 0.94952845]
 [8.46224055 4.52390408 1.46516322]
 [8.4457529  4.52285888 1.62605592]
 [8.47380078 4.41680695 1.34997206]
 [8.47136427 4.49714295 1.64130731]
 [8.46048545 4.42762024 0.99743797]
 [8.46745032 4.65099578 1.04299542]
 [8.46745913 4.64592978 1.51766867]
 [8.46969467 4.61318973 1.40776273]
 [8.52469016 6.54532788 1.26183272]
 [8.51963169 6.6399889  1.35131468]
 [8.53580431 6.72816142 2.18482242]
 [8.51881067 6.52801721 1.80512387]
 [8.53615155 6.55001817 1.59132179]
 [8.52596828 6.74200767 2.04251714]
 [8.53936962 6.73313478 1.41725194]
 [8.52184564 6.66116136 2.2151968 ]
 [8.52711601 6.75739247 2.19141941]
 [8.52590683 6.65103157 1.99667915]
 [8.54947624 6.51714107 3.54687297]
 [8.56087328 6.35250572 3.63651056]
 [8.54069076 6.35738788 3.65991261]
 [8.55113219 6.40442076 3.46425226]
 [8.53974812 6.49623725 3.66773129]
 [8.53711239 6.39239004 3.28227744]
 [8.55060601 6.3831375  3.64889309]
 [8.54809286 6.41507315 3.50

In [62]:
# process of classification
rf = RandomForestClassifier(oob_score = True)
rf.fit(x_basic, labels_basic)

# check the score for train-set
print("oob-score = ", rf.oob_score_)

oob-score =  1.0
